In [ ]:
##to install bigquery client library in python
#pip install --upgrade google-cloud-BigQuery
#conda install -c conda-forge google-cloud-bigquery google-cloud-bigquery-storage pandas pyarrow

## to upgrade pandas gbq
#conda install pandas-gbq --channel conda-forge
#!pip install --upgrade 'google-cloud-bigquery[bqstorage,pandas]'


In [3]:
#set GOOGLE_APPLICATION_CREDENTIALS=KEY_PATH
##keypath is the json key of service account saved on pc
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="C:/Users/kashi/Desktop/JUNO bootcamp info-Data analyst/junobigqueryclass-key.json"

In [4]:
%load_ext google.cloud.bigquery

# Item Info table

In [6]:
%%bigquery
SELECT *
FROM
  `junobigqueryclass.game_data.item_info`
LIMIT
  1

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.24s/rows]


,item_id,price
0,35d02644f8f74e818546a6803600dd8a,2.0


# Matches Info table

In [4]:
%%bigquery
SELECT *
FROM `junobigqueryclass.game_data.matches_info` 
LIMIT 1

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.09s/rows]


,player_id,match_id,opponent_id,outcome,day
0,650cf321ff3e4dec8d247854d62f0604,a226a425199c4494bd2a5fca48d85ea9,e2cdc18efc534b4bb740d14aa45d5f7a,win,1


# Player Info table

In [31]:
%%bigquery
SELECT *
FROM `junobigqueryclass.game_data.player_info` 


Downloading: 100%|██████████████████████████████████████████████████████████| 49510/49510 [00:01<00:00, 30102.30rows/s]


,player_id,location,age,system,joined
0,49d65bd7b1bd4d398e4bb67fe6fb6082,Asia,8,Linux,104
1,eeae10da5c1747b38b51bf860399af2c,Africa,8,Linux,272
2,86478665e3194a52bc453be87c7760fc,South America,9,Linux,128
3,49dab7705da64b31b590ece570fcfdee,South America,9,Linux,148
4,cd7a16dfe302454cad858da1ebba52c9,North America,9,Linux,254
...,...,...,...,...,...
49505,1f5699cb2363477d8c72116c4685edb2,Oceania,31,Windows,100
49506,465033bf98d74d12bff6c01776cc791f,Oceania,32,Windows,8
49507,1329f92c6ce74f12b1c003892be4e132,Africa,32,Windows,116
49508,138b9537be514488baa2d8026f73807a,Asia,32,Windows,245


# Purchase Info table

In [6]:
%%bigquery
SELECT *
FROM `junobigqueryclass.game_data.purchase_info` 
LIMIT 1

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.11s/rows]


,player_id,item_id,day
0,a2850b5e0b054d0a97338c14d0fd8338,54c53d6ef25c4f94af77e48c75f7e802,1


### The total number of players who joined on a given day

In [19]:
%%bigquery
SELECT joined as joining_day, 
COUNT(player_id) players_joined
FROM `junobigqueryclass.game_data.player_info` 
GROUP BY 1
ORDER BY 1

Downloading: 100%|████████████████████████████████████████████████████████████████| 365/365 [00:01<00:00, 344.20rows/s]


,joining_day,players_joined
0,1,121
1,2,158
2,3,95
3,4,154
4,5,126
...,...,...
360,361,106
361,362,182
362,363,83
363,364,119


## Finding Retained players on a particular day

#### 1. Last play day table

In [32]:
%%bigquery
SELECT player_id as player_lastday,
    MAX(day) as lastplay_day
FROM `junobigqueryclass.game_data.matches_info` 
GROUP BY 1

Downloading: 100%|██████████████████████████████████████████████████████████| 49365/49365 [00:01<00:00, 31899.66rows/s]


,player_lastday,lastplay_day
0,650cf321ff3e4dec8d247854d62f0604,30
1,49483dcbeefe4f0fa0aade55fa0e7e54,33
2,4acd268a7717437cbcc172f215ba640c,46
3,74e7570d816442c891719a8296acab5a,33
4,1f360517123f47e79800b12a88a7d6ed,51
...,...,...
49360,da9c369c84194cb4873f624b07586011,365
49361,0c98196ad466432c9bc6aaf465b8cbee,365
49362,db4ed21e12e142aba00dadae824e4503,365
49363,77f5391cc83b41c7a55116b07c683f04,365


#### 2. Joining lastplay table for each player 

In [33]:
%%bigquery
SELECT players.player_id,
    players.joined as joining_day,
    lastplay.lastplay_day
FROM `junobigqueryclass.game_data.player_info` players
JOIN 
(
SELECT player_id as player_lastday,
    MAX(day) as lastplay_day
FROM `junobigqueryclass.game_data.matches_info` 
GROUP BY 1
) as lastplay

ON players.player_id = lastplay.player_lastday

Downloading: 100%|██████████████████████████████████████████████████████████| 49365/49365 [00:01<00:00, 34763.46rows/s]


,player_id,joining_day,lastplay_day
0,49d65bd7b1bd4d398e4bb67fe6fb6082,104,127
1,eeae10da5c1747b38b51bf860399af2c,272,314
2,86478665e3194a52bc453be87c7760fc,128,165
3,49dab7705da64b31b590ece570fcfdee,148,198
4,cd7a16dfe302454cad858da1ebba52c9,254,275
...,...,...,...
49360,1f5699cb2363477d8c72116c4685edb2,100,133
49361,465033bf98d74d12bff6c01776cc791f,8,57
49362,1329f92c6ce74f12b1c003892be4e132,116,139
49363,138b9537be514488baa2d8026f73807a,245,287


#### 3. Getting the list of retained player from above table
##### condition : if a given player play a match 30 days after he or she joined is a retained player

In [35]:
%%bigquery
SELECT days_info.player_id as retainedplayer_id,
    days_info.joining_day
FROM
(SELECT players.player_id,
    players.joined as joining_day,
    lastplay.lastplay_day
FROM `junobigqueryclass.game_data.player_info` players
JOIN 
(
SELECT player_id as player_lastday,
    MAX(day) as lastplay_day
FROM `junobigqueryclass.game_data.matches_info` 
GROUP BY 1
) as lastplay

ON players.player_id = lastplay.player_lastday) as days_info

WHERE days_info.lastplay_day - days_info.joining_day > 30

Downloading: 100%|██████████████████████████████████████████████████████████| 30982/30982 [00:01<00:00, 24239.94rows/s]


,retainedplayer_id,joining_day
0,eeae10da5c1747b38b51bf860399af2c,272
1,86478665e3194a52bc453be87c7760fc,128
2,49dab7705da64b31b590ece570fcfdee,148
3,34c91fdb23004044848f9c05fa38fd31,29
4,327b96a94ed54b8cabbc1d15ce3f0342,77
...,...,...
30977,dfa5d8a2502844e58b745a353eb29313,294
30978,1f5699cb2363477d8c72116c4685edb2,100
30979,465033bf98d74d12bff6c01776cc791f,8
30980,138b9537be514488baa2d8026f73807a,245
